In [3]:
import json
import numpy as np
import pandas as pd
import csv
import gluonnlp as nlp
import multiprocessing as mp

## Load and merge the data
The source is Kaggle, [IMDB Spoiler Dataset](https://www.kaggle.com/rmisra/imdb-spoiler-dataset). The downloaded zip includes two files, we will demonstrate some of their contents below.

In [4]:
df_reviews = pd.read_json('./data/imdb/IMDB_reviews.json', lines=True).\
             drop_duplicates('review_text').sample(frac=1)
df_reviews.review_date = pd.to_datetime(df_reviews.review_date, infer_datetime_format=True)
df_reviews.user_id = df_reviews.user_id.astype('category')
print(df_reviews.shape)
review_fields = [field for field in df_reviews]
df_reviews.tail()

(573385, 7)


,is_spoiler,movie_id,rating,review_date,review_summary,review_text,user_id
410289,False,tt0396171,4,2007-11-29,Don't let this film prevent you from reading t...,The novel by Patrick Süskind on which the scri...,ur17559125
510061,False,tt0110148,4,2002-01-22,"Apparently, Vampires Don't Fear CHEESE, Either!!!",Let me begin by saying that I have never read ...,ur1542448
312153,False,tt0289043,3,2003-04-27,It's all been done before,"This story has been done so many times, it's h...",ur2347354
182045,False,tt1663662,8,2013-07-13,Cancelling the apocalypse has never looked so ...,The publicity for Guillermo del Toro's robots ...,ur22164449
491546,True,tt0102526,9,2017-05-28,Wesley Snipes is brilliant,This movie is unlike any other gangster movie ...,ur33697153


In [5]:
# the total number of spoilers in dataset
num_spoilers = sum(df_reviews.is_spoiler)
print('num of spoilers in all: %d\nration of spoilers in all: %.4f\na dummy classifier can achieve acc of %.4f' \
      % (num_spoilers, num_spoilers / df_reviews.shape[0], \
         1 - num_spoilers / df_reviews.shape[0]))
# only less than 20% of the user comments' explicitly say that they contain spoilers
num_notice = df_reviews.review_text.str.contains('spoiler', case=False).sum() / \
             df_reviews.is_spoiler.sum()
print('a rough estimation of how many users declare their reviews as spoilers: %.4f' % num_notice)

num of spoilers in all: 150856
ration of spoilers in all: 0.2631
a dummy classifier can achieve acc of 0.7369
a rough estimation of how many users declare their reviews as spoilers: 0.1752


In [6]:
tokenizer = nlp.data.SpacyTokenizer('en')
def get_word_count(x):
    return len(tokenizer(x))

with mp.Pool() as pool:
    length_list = pool.map(get_word_count, df_reviews.review_text)

In [11]:
print('the average number of words in a review is:', round(sum(length_list) / df_reviews.shape[0]))

the average number of words in a review is: 303


In [13]:
# imdb information about the movies
df_movies = pd.read_json('./data/imdb/IMDB_movie_details.json', lines=True)
df_movies.release_date = pd.to_datetime(df_movies.release_date, infer_datetime_format=True)
print(df_movies.shape)
df_movies.tail()

(1572, 7)


,duration,genre,movie_id,plot_summary,plot_synopsis,rating,release_date
1567,1h 53min,"[Sci-Fi, Thriller]",tt0289879,Evan Treborn grows up in a small town with his...,"In the year 1998, Evan Treborn (Ashton Kutcher...",7.7,2004-01-23
1568,1h 41min,[Drama],tt1723811,Brandon is a 30-something man living in New Yo...,"Brandon (Michael Fassbender) is a successful, ...",7.2,2012-01-13
1569,1h 46min,"[Action, Drama, History]",tt5013056,Evacuation of Allied soldiers from the British...,The film alternates between three different pe...,8.1,2017-07-21
1570,1h 33min,"[Comedy, Drama]",tt0104014/,"For a while now, beautiful 24-year-old Diana B...",,5.3,1992-02-21
1571,1h 32min,"[Drama, Thriller]",tt0114142/,"The marriage of David Burgess, a senior execut...",,4.0,1999-01-29


In [14]:
with mp.Pool() as mp:
    length_list = mp.map(get_word_count, df_movies.plot_summary)
print('the average number of words in a plot summary is %d' % \
      round(sum(length_list) / df_movies.shape[0]))

the average number of words in a plot summary is 120


In [15]:
# join the two dataframe according to movie id
df_reviews = df_reviews.merge(df_movies, on="movie_id",how="left",suffixes=('_review','_movie'))

In [16]:
columns_keep = ['is_spoiler', 'movie_id', 'plot_summary', 'review_summary', 'review_text',]
all_columns = [review for review in df_reviews]
for x in columns_keep:
    all_columns.remove(x)
drop_columns = all_columns
df_train = df_reviews.drop(columns=drop_columns)
df_train['review_text'] = df_reviews['review_summary']+ ' ' + df_reviews['review_text']
df_train = df_train.drop(columns=['review_summary'])
df_train.tail()

,is_spoiler,movie_id,review_text,plot_summary
573380,False,tt0396171,Don't let this film prevent you from reading t...,Jean-Baptiste Grenouille came into the world u...
573381,False,tt0110148,"Apparently, Vampires Don't Fear CHEESE, Either...",It hasn't even been a year since a plantation ...
573382,False,tt0289043,It's all been done before This story has been ...,Animal activists invade a laboratory with the ...
573383,False,tt1663662,Cancelling the apocalypse has never looked so ...,"When monstrous creatures, known as Kaiju, star..."
573384,True,tt0102526,Wesley Snipes is brilliant This movie is unlik...,The gangster Nino has a gang who call themselv...


## Save the processed data for future use

In [17]:
df_train.to_csv('./data/train.csv', sep='|', index=False)